# NYC Flights 2013 Analysis

In [1]:
install.packages("nycflights13")
library(nycflights13)
library("tidyverse")
library(tidyverse, warn.conflicts=FALSE)
options(dplyr.summarise.inform = FALSE)

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
Warning message:
“Failed to locate timezone database”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.1     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()  masks stats::filter()
✖ purrr::flatten() masks jsonlite::flatten()
✖ dplyr::lag()     masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
data(package="nycflights13")

Data sets 
 
 A data.frame: 5 × 3 
 
 Package Item Title 
 <chr> <chr> <chr> 
 
 
 nycflights13 airlines Airline names. 
 nycflights13 airports Airport metadata 
 nycflights13 flights Flights data 
 nycflights13 planes Plane metadata. 
 nycflights13 weather Hourly weather data

In [3]:
glimpse(flights)
cat("\n")
glimpse(airlines)
cat("\n")
glimpse(airports)
cat("\n")
glimpse(weather)
cat("\n")
glimpse(planes)
cat("\n")

Rows: 336,776
Columns: 19
$ year           <int> 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2…
$ month          <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ day            <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ dep_time       <int> 517, 533, 542, 544, 554, 554, 555, 557, 557, 558, 558, …
$ sched_dep_time <int> 515, 529, 540, 545, 600, 558, 600, 600, 600, 600, 600, …
$ dep_delay      <dbl> 2, 4, 2, -1, -6, -4, -5, -3, -3, -2, -2, -2, -2, -2, -1…
$ arr_time       <int> 830, 850, 923, 1004, 812, 740, 913, 709, 838, 753, 849,…
$ sched_arr_time <int> 819, 830, 850, 1022, 837, 728, 854, 723, 846, 745, 851,…
$ arr_delay      <dbl> 11, 20, 33, -18, -25, 12, 19, -14, -8, 8, -2, -3, 7, -1…
$ carrier        <chr> "UA", "UA", "AA", "B6", "DL", "UA", "B6", "EV", "B6", "…
$ flight         <int> 1545, 1714, 1141, 725, 461, 1696, 507, 5708, 79, 301, 4…
$ tailnum        <chr> "N14228", "N24211", "N619AA", "N804JB", "N668DN", "N394…
$ origin      

## Q1: Which airlines had the least departure delay at the beginning of 2013?

In [4]:
flights %>%
    filter(year == 2013, month == 1, day == 1) %>%
    select(carrier, dep_delay) %>%
    arrange(dep_delay) %>%
    head(5) %>%
    inner_join(airlines, by = "carrier")

A tibble: 5 × 3 
 
 carrier dep_delay name 
 <chr> <dbl> <chr> 
 
 
 MQ -15 Envoy Air 
 AA -15 American Airlines Inc. 
 F9 -14 Frontier Airlines Inc. 
 EV -13 ExpressJet Airlines Inc. 
 B6 -12 JetBlue Airways

## Q2: Which destinations did people travel to the most during the summer (July, August, and September) of 2013?

In [5]:
flights %>%
    filter(!is.na(dep_time), year == 2013, month %in% c(7,8,9),) %>%
     select(year, month, dest) %>%
     group_by(year, month, dest) %>% 
     summarise(n_filghts = n()) %>%
     arrange(desc(n_filghts)) %>%
     head(5) %>%
     inner_join(airports %>%
        select(faa, name),
        by = c("dest" = "faa"))

A grouped_df: 5 × 5 
 
 year month dest n_filghts name 
 <int> <int> <chr> <int> <chr> 
 
 
 2013 8 ORD 1571 Chicago Ohare Intl 
 2013 9 ORD 1517 Chicago Ohare Intl 
 2013 8 LAX 1502 Los Angeles Intl 
 2013 7 ORD 1499 Chicago Ohare Intl 
 2013 7 LAX 1497 Los Angeles Intl

## Q3: Which airlines had the most canceled flights in 2013?

In [6]:
deptime <- flights %>%                                   
  filter(is.na(dep_time)) %>%                               
  group_by(carrier) %>%                                     
  summarise(canceled_flights = n())                         

flights %>% 
  inner_join(airlines, by = "carrier") %>%                  
  group_by(carrier, name) %>% 
  select(carrier, name) %>%                                 
  summarise(scheduled_flights = n()) %>%                    
  inner_join(deptime, by = "carrier") %>%                
  mutate(canceled_flights_proportion =                      
      round((canceled_flights/scheduled_flights), 3)) %>% 
  arrange(desc(canceled_flights_proportion)) %>%            
  head(5)                     

A grouped_df: 5 × 5 
 
 carrier name scheduled_flights canceled_flights canceled_flights_proportion 
 <chr> <chr> <int> <int> <dbl> 
 
 
 OO SkyWest Airlines Inc. 32 3 0.094 
 YV Mesa Airlines Inc. 601 56 0.093 
 9E Endeavor Air Inc. 18460 1044 0.057 
 EV ExpressJet Airlines Inc. 54173 2817 0.052 
 MQ Envoy Air 26397 1234 0.047

## Q4: What are the fastest flights compared to the distance traveled?

In [7]:
flights %>% 
  select(carrier, air_time, distance) %>%                   
  mutate(distance_interval = case_when(                     
    distance <= 900 ~ "less than 900 miles",
    distance <= 1800 ~ "900 - 1800 miles",
    TRUE ~ "more than 1800 miles"
  )) %>% 
  group_by(carrier, distance_interval) %>%                  
  summarise(min_air_time = min(air_time, na.rm = T)) %>%    
  arrange(carrier, min_air_time) %>%
  head(10)

A grouped_df: 10 × 3 
 
 carrier distance_interval min_air_time 
 <chr> <chr> <dbl> 
 
 
 9E less than 900 miles 21 
 9E 900 - 1800 miles 127 
 AA less than 900 miles 29 
 AA 900 - 1800 miles 116 
 AA more than 1800 miles 264 
 AS more than 1800 miles 277 
 B6 less than 900 miles 29 
 B6 900 - 1800 miles 110 
 B6 more than 1800 miles 212 
 DL less than 900 miles 26

## Q5: Did airport in NYC have the same amount of flights provided?

In [8]:
airportname <- airports %>%
  select(faa, name)

nycairport <- flights %>%
group_by(origin) %>%
    summarise(mean_distance = round(mean(distance), 3),
            flights = n()) %>%
left_join(airportname, by = c("origin" = "faa"))

nycairport

A tibble: 3 × 4 
 
 origin mean_distance flights name 
 <chr> <dbl> <int> <chr> 
 
 
 EWR 1056.743 120835 Newark Liberty Intl 
 JFK 1266.249 111279 John F Kennedy Intl 
 LGA 779.836 104662 La Guardia